In [ ]:
import pandas as pd

In [ ]:
#IMP POINTS TO KEEP IN MIND TO RUN THIS CODE
#Keep cik_ticker.csv file in the directory
#Flow of the code -- 1. All the earning call links are extracted and stored in a article_links_final.txt file
#                    2. The links and their following attributes are stored in a earnings_call_links_final.csv file
#                    3. The earning transcripts are saved in the article folder of the directory

In [ ]:
db=pd.read_csv('cik_ticker.csv')
db_ticker=db[db['TICKERH'].notnull()]
db_ticker.drop_duplicates()
data=db_ticker['TICKERH'].unique()
data=pd.DataFrame(data,columns=['TICKERH'])

In [ ]:
import json, requests, codecs, time
import pandas as pd

# Function to write data to a file
def write_to_file(file_name, data):
    with codecs.open("./" + file_name, "a", "utf-8") as f:
        f.write(data)

# Function to read data from a file
def read_file(file_name):
    data = list()
    with codecs.open("./" + file_name, encoding='utf-8') as f:
        for line in f:
            result = line.rstrip('\r').rstrip('\n').split('\t')
            data.append(result)
    return data

# Function to make a request and process the response
def response(ticker, page_num):
    print(ticker, page_num)

    time.sleep(5)  # Delay to avoid hitting API rate limits

    main_url = 'https://seekingalpha.com/api/v3/symbols/{0}/transcripts?id={0}&page[size]=50&page[number]={1}'
    response_text = requests.get(url=main_url.format(ticker, page_num), headers=headers).text
    json_response = json.loads(response_text)

    if page_num == 1:
        total_pages = json_response['meta']['page']['totalPages']

        # Recursively call the response function for remaining pages
        for i in range(2, total_pages + 1):
            response(ticker, i)

    publishOn = ''
    title = ''
    links = ''

    # Extract relevant information from the response and write to file
    for articles in json_response['data']:
        publishOn = articles['attributes']['publishOn']
        title = articles['attributes']['title']
        links = articles['links']['self']

        write_to_file('./articles_links_final.txt', '%s\t%s\t%s\t%s\n' % (ticker, publishOn, title, links))

headers = {
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
    'cookie': 'machine_cookie=4397208011929; _pprv={"consent":{"0":{"mode":"opt-in"},"1":{"mode":"opt-in"},"2":{"mode":"opt-in"},"3":{"mode":"opt-in"},"4":{"mode":"opt-in"},"5":{"mode":"opt-in"},"6":{"mode":"opt-in"},"7":{"mode":"opt-in"}}}; _pcid={"browserId":"l7hnvrk0k6wpdlva"}; __pat=-14400000; session_id=07697a1e-60c8-4f07-87f1-4006dce49a5e; user_id=48681666; user_nick=russian_guy2010; user_devices=1; u_voc=44; marketplace_author_slugs=; user_cookie_key=1c5kbnu; has_paid_subscription=false; user_perm=; sapu=101; user_remember_token=e9c43ed5921d366c110a2ca402d4a9b9575a0ce6; gk_user_access=1**1662571938; gk_user_access_sign=2d8e89fcaf1992528d5a1d63360ced8acf4f9ad2; pxcts=09b9bc9e-2ed3-11ed-b3c3-4a54764c7561; _pxvid=09b9aeb0-2ed3-11ed-b3c3-4a54764c7561; _sasource=; __tac=; __tae=1662571963924; LAST_VISITED_PAGE={"pathname":"https://seekingalpha.com/article/4461908-facebook-inc-fb-ceo-mark-zuckerberg-on-q3-2021-results-earning-call-transcript","pageKey":"6cec6c2a-742a-416d-8915-0faae0c93259"}; _pctx={u}N4IgDghg5gpgagSxgdwJIBMQC4QBsDsALvmABwB2YAblAJ4BsAVgPboBGIANCAK4DOMAE59s5Hrlzd+QgMqEIhfthARyzclxB8EhGBlHjcAXyA; __pvi={"id":"v-2022-09-08-09-44-21-168-I9yLlDK6zDuGXfcH-62b77c9904f8d4be970fc2fad5c9ab40","domain":".seekingalpha.com","time":1662651376414}; __tbc={kpex}L9BJfoJlsxU92bbMefDXzlfE46uyHUEnwyvxN1Ac_Nd0m1J1BLZiJc7xKbDOeKnC; xbc={kpex}7U3ogJCz-qcJS2lhvFzxt1PzipHTwrj51aUqALimTiIp-omNegRWXKocnStSWcYRemVK9MjWk4G7NV8wArgwZ34PSNV_lQhjgVL9CzHMOsjLZbbY9EZG2bau6UXTl875E0VOsfjwEW9FJcQapBp0AYh0eaWyIAFf2eHOYIlACSA8lzYZoNFBUgqbtkOBbghJ1jtRHdw0l8ofD3ZCrWLB5NkNRSHMzFNyYMCZe7R6_VXtVqOt1KJWC1c3AZo-X8fr971m8imQD0JcUghlN0qZjwGfuj1tcdybY_4X3jfY97qqCsk7lT2K2Fvz1ea-khxDfsc2dWQ-uq7pbeHBtwZ_YY89a_wBuQp2G26A6SWP8p7RpRxNHeUOAYpe76g5qkgUu2LSWtCQ03pIZa39EeEfzJ31UZpHjD3ZjWqo8K-Tc__iWHfR-9jChyU9HJynswDAQ5jDCW9QjnnzhKrGYAcjP0MN9tw-SwB4A-I5JN_y-1bsXnY2T8sgFDIa900Vi6rHSOnt_rum-59eN076xRhXD2XuP9Vu5jz98Pifa3Ax6sysQ4d9kmJ3hntgmzZN7FrvlCsax8EhKu3xQJObmTQlY32rMte_K-VJv_qx1yuLZPXSo5ZFP6jyAr34Q0IIvJs751cRdoFZ5o7byW5cQA6WylzFO1XVYZcSYYZfMvZ1ghYUlXkB1ACwGZByp-CIQSxF2a133CmKVMGgP8m3EmTXMr5NRgXq-imsUuLx11q2WW49yPT_40HoEh2RvPs_d4IK',
    'Host': 'seekingalpha.com'
}


stock_tickers = data

# Iterate over each row in the DataFrame
for index, row in stock_tickers.iterrows():
    ticker = row['TICKERH']

    try:
        response(ticker, 1)
    except:
        pass


The code reads the contents of the "articles_links_final.txt" file and processes each line. It splits the lines based on tab ('\t') delimiter and extracts the required information such as ticker, date, earning call links, company name, and the original line. These extracted values are then appended as lists to the 'rows' list. Finally, a DataFrame is created using the 'rows' data and column names, and it can be saved as a CSV file

In [ ]:
import pandas as pd

# Read the text file
with open('articles_links_final.txt', 'r') as file:
    data = file.readlines()

# Define the column names
columns = ['ticker', 'earning_call_links', 'company name', 'date', 'links']

# Initialize an empty list to store the data
rows = []

# Process each line in the text file
for line in data:
    links = line  # Store the original line as 'links' variable
    line = line.strip().split('\t')  # Remove leading/trailing whitespace and split the line by tabs
    ticker = line[0]  # Extract the ticker from the first column
    date = line[1].split('T')[0]  # Extract the date from the second column and remove the time part
    earning_call_links = line[3]  # Extract the earning call links from the fourth column
    company_name = line[2].split(' (')[0]  # Extract the company name from the third column and remove additional information
    rows.append([ticker, earning_call_links, company_name, date, links])  # Append the extracted values as a list to 'rows'

# Create a DataFrame from the data
df = pd.DataFrame(rows, columns=columns)

# Save the DataFrame as a CSV file
df.to_csv('earnings_call_links_final.csv', index=False)



In [ ]:
folder_name = "article"
directory = "./"

folder_path = os.path.join(directory, folder_name)

# Check if the folder already exists
if not os.path.exists(folder_path):
    # Create the folder
    os.makedirs(folder_path)

In [ ]:
import requests, codecs, time
from lxml import etree
import pandas as pd
import os.path

This code reads stock ticker data from a CSV file and iterates over each row. For each row, it retrieves an article link and checks if a corresponding text file exists. If the file doesn't exist, it makes a request to the article URL, extracts the content, and writes it to a text file.

In [ ]:
import codecs
import requests
from lxml import etree
import os
import pandas as pd

# Function to write data to a file
def write_to_file(file_name, data):
    with codecs.open("./" + file_name, "w", "utf-8") as f:
        f.write(data)

# Function to read data from a file
def read_file(file_name):
    data = list()
    with codecs.open("./" + file_name, encoding='utf-8') as f:
        for line in f:
            result = line.rstrip('\r').rstrip('\n').split('\t')
            data.append(result)
    return data

# Function to process the response and write content to a file
def response(article):
    print(article)
    main_url = 'https://seekingalpha.com{0}'
    response_text = requests.get(url=main_url.format(article), headers=headers).text
    tree = etree.HTML(response_text)
    final_text = ''
    for paragraph in tree.xpath("//div[@data-test-id='article-content']//p"):
        final_text += ''.join(paragraph.itertext()) + '\n'
    write_to_file('.' + article + '.txt', final_text)

headers = {
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
    'cookie': '_sasource=; session_id=252127c0-f660-43a1-9ec8-cc600d61806a; machine_cookie=4397208011929; _pprv={"consent":{"0":{"mode":"opt-in"},"1":{"mode":"opt-in"},"2":{"mode":"opt-in"},"3":{"mode":"opt-in"},"4":{"mode":"opt-in"},"5":{"mode":"opt-in"},"6":{"mode":"opt-in"},"7":{"mode":"opt-in"}}}; _pcid={"browserId":"l7hnvrk0k6wpdlva"}; __pat=-14400000; user_id=48681666; user_nick=russian_guy2010; user_devices=1; u_voc=44; marketplace_author_slugs=; user_cookie_key=1c5kbnu; has_paid_subscription=false; user_perm=; sapu=101; user_remember_token=e9c43ed5921d366c110a2ca402d4a9b9575a0ce6; pxcts=09b9bc9e-2ed3-11ed-b3c3-4a54764c7561; _pxvid=09b9aeb0-2ed3-11ed-b3c3-4a54764c7561; _sasource=; gk_user_access=1**1662671240; gk_user_access_sign=d1f4ec9011b95c429122e97a95318cc899ee614b; xbc={kpex}oT8V1U7iFe7hpRD4GWbYrGdxhqYSkCjh5CDfaA_R-t_NUFp-RU7zKtQYJk4V-9v-4Lg2gFbQSqDAhPsPoSHw3bhmriRUjj_XC-p5sKncXiHMR4Jsqjgk5mHEMQ1IkT-pA8cGcTi__CJGWdRfSwheUh1tFX7v5AoY2Z_gJ22BTgM8lzYZoNFBUgqbtkOBbghJPHsktcC0HwsMCrBXJIDZczKqgObCKs2mkXUDQHUFh33tVqOt1KJWC1c3AZo-X8fr2cb3teJDeTW9r_bgmQZ0RgGfuj1tcdybY_4X3jfY97rh0wnjKcxRoEQobC8bLl7Kfsc2dWQ-uq7pbeHBtwZ_YY89a_wBuQp2G26A6SWP8p7RpRxNHeUOAYpe76g5qkgUu2LSWtCQ03pIZa39EeEfzJ31UZpHjD3ZjWqo8K-Tc__iWHfR-9jChyU9HJynswDAQ5jDCW9QjnnzhKrGYAcjP0MN9tw-SwB4A-I5JN_y-1bsXnY2T8sgFDIa900Vi6rHSOnt_rum-59eN076xRhXD2XuP9Vu5jz98Pifa3Ax6sysQ4d9kmJ3hntgmzZN7FrvlCsax8EhKu3xQJObmTQlY32rMte_K-VJv_qx1yuLZPXSo5ZFP6jyAr34Q0IIvJs751cRdoFZ5o7byW5cQA6WylzFO1XVYZcSYYZfMvZ1ghYUlXkB1ACwGZByp-CIQSxF2a133CmKVMGgP8m3EmTXMr5NRgXq-imsUuLx11q2WW49yPT_40HoEh2RvPs_d4IK; session_id=6370d439-b997-455d-83b6-d79d7d5bff1c; LAST_VISITED_PAGE={"pathname":"https://seekingalpha.com/","pageKey":"ecbd1da5-31d6-4a6f-ad47-e9aa800696ad"}; _pctx={u}N4IgDghg5gpgagSxgdwJIBMQC4QBsDsArggG7IAMALGAMY0AcJElNIANCIQM4wBOX2AHaFcuDtz4BlAC4Rp3bCAiCA9oPYguCaTAxCRuAL5A; __pvi={"id":"v-2022-09-09-08-37-08-555-09bHVrbwyN6f8b6o-a0d682c333a84b455511e31242b21873","domain":".seekingalpha.com","time":1662730628555}; __tac=; __tae=1662730628721; __tbc={kpex}L9BJfoJlsxU92bbMefDXzlfE46uyHUEnwyvxN1Ac_Nd0m1J1BLZiJc7xKbDOeKnC',
    'Host': 'seekingalpha.com'
}

# Read the stock tickers from a CSV file
stock_tickers = pd.read_csv('./earnings_call_links_final.csv')

# Iterate over each row in the DataFrame
for index, row in stock_tickers.iterrows():
    article = row['earning_call_links']

    try:
        if not os.path.isfile('.' + article + '.txt'):
            response(article)
    except:
        pass

# Perform the same for each line in the CSV file
